### Input files: Price-Today, orders-log
### Output files: Price-for-Order, orders-log, monthly-sales

In [2]:
import glob
import os
import pandas as pd
from datetime import date, timedelta, datetime
from pandas.tseries.offsets import *
from sqlalchemy import create_engine

engine = create_engine("sqlite:///c:\\ruby\\port_lite\\db\\development.sqlite3")
conlite = engine.connect()
engine = create_engine('mysql+pymysql://root:@localhost:3306/portfolio_development')
conpf = engine.connect()

format_dict = {'qty':'{:,}',
              'sell_price':'{:,.2f}','buy_price':'{:,.2f}','diff':'{:,.2f}', 'target':'{:,.2f}',
              'cost':'{:,.2f}','price':'{:,.2f}','%change':'{:,.2f}','%profit':'{:,.2f}',
              'sell_amt':'{:,.2f}','buy_amt':'{:,.2f}','gross':'{:,.2f}','amount':'{:,.2f}',
              'cst_amt':'{:,.2f}','mkt_amt':'{:,.2f}','difa':'{:,.2f}',  
              'pct':'{:,.2f}','net':'{:,.2f}'}

pd.options.display.max_rows = 50
pd.options.display.float_format = '{:.2f}'.format

today = date.today()
yesterday = today - timedelta(days=1)
print(today, yesterday)

2025-08-28 2025-08-27


In [3]:
num_business_days = BDay(1)
yesterday = today - num_business_days
print(f'today: {today}')
yesterday = yesterday.date()
print(f'yesterday: {yesterday}')

today: 2025-08-28
yesterday: 2025-08-27


In [4]:
# If run after actual work day
#today = today - timedelta(days=2)
#print(today)

In [5]:
# Get the current time
current_time = datetime.now()
# Format the time to remove milliseconds
formatted_time = current_time.strftime("%Y-%m-%d %H:%M:%S")
print(formatted_time)

2025-08-28 21:01:14


### Restart and Run All Cells

In [7]:
# Get the user's home directory
user_path = os.path.expanduser('~')
# Get the current working directory
current_path = os.getcwd()
# Derive the base directory (base_dir) by removing the last folder ('Daily')
base_path = os.path.dirname(current_path)
#C:\Users\PC1\OneDrive\A5\Data
dat_path = os.path.join(base_path, "Data")
#C:\Users\PC1\OneDrive\A5\Ruby
ruby_path = os.path.join(base_path, "Ruby")
#C:\Users\PC1\OneDrive\Imports\santisoontarinka@gmail.com - Google Drive\Data>
god_path = os.path.join(user_path, "OneDrive","Imports","santisoontarinka@gmail.com - Google Drive","Data")
#C:\Users\PC1\iCloudDrive\data
icd_path = os.path.join(user_path, "iCloudDrive", "Data")
#C:\Users\PC1\OneDrive\Documents\obsidian-git-sync\Data
osd_path = os.path.join(user_path, "OneDrive","Documents","obsidian-git-sync","Data")

In [8]:
print("User path:", user_path)
print(f"Current path: {current_path}")
print(f"Base path: {base_path}")
print(f"Data path (dat_path): {dat_path}") 
print(f"Ruby path (ruby_path): {ruby_path}") 
print(f"Google Drive path (god_path): {god_path}")
print(f"iCloudDrive path (icd_path): {icd_path}") 
print(f"Obsidian path (osd_path): {osd_path}") 

User path: C:\Users\PC1
Current path: C:\Users\PC1\OneDrive\A5\Daily
Base path: C:\Users\PC1\OneDrive\A5
Data path (dat_path): C:\Users\PC1\OneDrive\A5\Data
Ruby path (ruby_path): C:\Users\PC1\OneDrive\A5\Ruby
Google Drive path (god_path): C:\Users\PC1\OneDrive\Imports\santisoontarinka@gmail.com - Google Drive\Data
iCloudDrive path (icd_path): C:\Users\PC1\iCloudDrive\Data
Obsidian path (osd_path): C:\Users\PC1\OneDrive\Documents\obsidian-git-sync\Data


## Must rerun whenever there is change in orders

In [10]:
sql = """
SELECT trade, name, qty, price AS target, active, reason, market, xdate 
FROM orders
ORDER BY trade, name"""
orders = pd.read_sql(sql, conlite)
orders.shape

(16, 8)

In [11]:
file_name = "Price-Today.csv"
input_file = os.path.join(dat_path, file_name)
print(f"Input file: {input_file}")

Input file: C:\Users\PC1\OneDrive\A5\Data\Price-Today.csv


In [12]:
prices = pd.read_csv(input_file)
prices.shape

(169, 10)

In [13]:
df_merge  = pd.merge(orders, prices, on = 'name', how='inner')
df_merge.shape

(16, 17)

In [14]:
df_merge['spd'] = 0

In [15]:
colo = ['trade', 'name', 'spd', 'reason', 'market', 'qty', 'target', 'price', 'change', '%change',
        'active', 'xdate']
df_merge[colo].shape

(16, 12)

In [16]:
file_name = "Price-for-Order.csv"
output_file = os.path.join(dat_path, file_name)
god_file = os.path.join(god_path, file_name)
icd_file = os.path.join(icd_path, file_name)
osd_file = os.path.join(osd_path, file_name)

In [17]:
print(f"Output file: {output_file}") 
print(f"icd_file : {icd_file}") 
print(f"god_file : {god_file}") 
print(f"osd_file : {osd_file}") 

Output file: C:\Users\PC1\OneDrive\A5\Data\Price-for-Order.csv
icd_file : C:\Users\PC1\iCloudDrive\Data\Price-for-Order.csv
god_file : C:\Users\PC1\OneDrive\Imports\santisoontarinka@gmail.com - Google Drive\Data\Price-for-Order.csv
osd_file : C:\Users\PC1\OneDrive\Documents\obsidian-git-sync\Data\Price-for-Order.csv


In [18]:
df_merge[colo].to_csv(output_file, header=True, index=False)
df_merge[colo].to_csv(god_file, header=True, index=False)
df_merge[colo].to_csv(icd_file, header=True, index=False)
df_merge[colo].to_csv(osd_file, header=True, index=False)

### call ruby ruby\order-log-new.rb

In [20]:
%pwd

'C:\\Users\\PC1\\OneDrive\\A5\\Daily'

In [21]:
os.chdir(base_path)
%pwd

'C:\\Users\\PC1\\OneDrive\\A5'

In [22]:
!ruby ruby\\order-log-new.rb

trade,name,spd,reason,market,qty,target,current,change,%change,active,xdate
B,AIMIRT,-12,RND15pct,SET,2500,9.1,9.7,0.0,0.0,1,2025-08-19
B,CPF,2,,SET50,5000,24.1,23.9,0.0,0.0,1,2025-08-29
B,GVREIT,0,RND20Pct,SET,5000,6.2,6.2,0.0,0.0,1,2025-08-27
B,JMART,-9,RD80Pct,SET50,3200,8.0,8.45,0.1,1.197605,1,2025-05-09
B,JMT,-9,RD75pct,SET50,2800,11.0,11.9,0.1,0.847458,1,2025-08-26
B,ORI,0,RD70Pct,SET100,5000,2.2,2.2,0.0,0.0,1,2025-03-13
B,PTT,-2,,SET50,2500,31.0,31.5,-0.25,-0.787402,1,2025-03-06
B,RCL,0,RD35pct,SET100,3000,28.0,28.0,-0.25,-0.884956,1,2025-08-22
B,TFFIF,1,RD30Pct,SET,10000,6.0,5.95,-0.05,-0.833333,1,2025-08-28
B,WHAIR,-46,RD50pct,SET,10000,4.3,5.55,0.05,0.909091,0,2025-08-18
B,WHART,-23,RD30Pct,SET,5000,8.0,9.15,0.0,0.0,0,2025-08-18
S,DIF,2,RD40Pct,SET,5000,8.3,8.2,0.0,0.0,1,2025-08-15
S,IVL,21,CSSTGT,SET50,1000,24.0,21.9,-0.3,-1.351351,0,2025-08-27
S,MCS,9,20Pct,SET,6000,8.75,8.3,0.05,0.606061,1,2025-08-26
S,NER,11,CSSMIN,SET,5000,4.5,4.28,0.02,0.469484,0,2025-08-22
S,PTG,59,COS

In [23]:
os.chdir(current_path)
%pwd

'C:\\Users\\PC1\\OneDrive\\A5\\Daily'

In [24]:
file_name   = 'orders-log.csv'
input_file = os.path.join(dat_path, file_name)
print(f"Input file: {input_file}")

Input file: C:\Users\PC1\OneDrive\A5\Data\orders-log.csv


In [25]:
df = pd.read_csv(input_file, sep=',', index_col=None)
df['amount'] = df['qty'] * df['target']
df.query('spd >= -3 and spd <= 3')

,trade,name,spd,reason,market,qty,target,current,change,%change,active,xdate,amount
1,B,CPF,2,NaN,SET50,5000,24.10,23.90,0.00,0.00,1,2025-08-29,120500.00
2,B,GVREIT,0,RND20Pct,SET,5000,6.20,6.20,0.00,0.00,1,2025-08-27,31000.00
5,B,ORI,0,RD70Pct,SET100,5000,2.20,2.20,0.00,0.00,1,2025-03-13,11000.00
6,B,PTT,-2,NaN,SET50,2500,31.00,31.50,-0.25,-0.79,1,2025-03-06,77500.00
7,B,RCL,0,RD35pct,SET100,3000,28.00,28.00,-0.25,-0.88,1,2025-08-22,84000.00
8,B,TFFIF,1,RD30Pct,SET,10000,6.00,5.95,-0.05,-0.83,1,2025-08-28,60000.00
11,S,DIF,2,RD40Pct,SET,5000,8.30,8.20,0.00,0.00,1,2025-08-15,41500.00


In [26]:
df.query('active == 2')

,trade,name,spd,reason,market,qty,target,current,change,%change,active,xdate,amount


In [27]:
df.query('active == 1')

,trade,name,spd,reason,market,qty,target,current,change,%change,active,xdate,amount
0,B,AIMIRT,-12,RND15pct,SET,2500,9.10,9.70,0.00,0.00,1,2025-08-19,22750.00
1,B,CPF,2,NaN,SET50,5000,24.10,23.90,0.00,0.00,1,2025-08-29,120500.00
2,B,GVREIT,0,RND20Pct,SET,5000,6.20,6.20,0.00,0.00,1,2025-08-27,31000.00
3,B,JMART,-9,RD80Pct,SET50,3200,8.00,8.45,0.10,1.20,1,2025-05-09,25600.00
4,B,JMT,-9,RD75pct,SET50,2800,11.00,11.90,0.10,0.85,1,2025-08-26,30800.00
5,B,ORI,0,RD70Pct,SET100,5000,2.20,2.20,0.00,0.00,1,2025-03-13,11000.00
6,B,PTT,-2,NaN,SET50,2500,31.00,31.50,-0.25,-0.79,1,2025-03-06,77500.00
7,B,RCL,0,RD35pct,SET100,3000,28.00,28.00,-0.25,-0.88,1,2025-08-22,84000.00
8,B,TFFIF,1,RD30Pct,SET,10000,6.00,5.95,-0.05,-0.83,1,2025-08-28,60000.00
11,S,DIF,2,RD40Pct,SET,5000,8.30,8.20,0.00,0.00,1,2025-08-15,41500.00


In [28]:
df.query('active == 0')

,trade,name,spd,reason,market,qty,target,current,change,%change,active,xdate,amount
9,B,WHAIR,-46,RD50pct,SET,10000,4.30,5.55,0.05,0.91,0,2025-08-18,43000.00
10,B,WHART,-23,RD30Pct,SET,5000,8.00,9.15,0.00,0.00,0,2025-08-18,40000.00
12,S,IVL,21,CSSTGT,SET50,1000,24.00,21.90,-0.30,-1.35,0,2025-08-27,24000.00
14,S,NER,11,CSSMIN,SET,5000,4.50,4.28,0.02,0.47,0,2025-08-22,22500.00
15,S,PTG,59,COS,SET100,3600,11.40,7.70,-0.10,-1.28,0,2025-03-10,41040.00


In [29]:
df.query('spd <= -40 and trade == "B"')

,trade,name,spd,reason,market,qty,target,current,change,%change,active,xdate,amount
9,B,WHAIR,-46,RD50pct,SET,10000,4.30,5.55,0.05,0.91,0,2025-08-18,43000.00


In [30]:
df.query('trade == "B" & spd > 0')

,trade,name,spd,reason,market,qty,target,current,change,%change,active,xdate,amount
1,B,CPF,2,NaN,SET50,5000,24.10,23.90,0.00,0.00,1,2025-08-29,120500.00
8,B,TFFIF,1,RD30Pct,SET,10000,6.00,5.95,-0.05,-0.83,1,2025-08-28,60000.00


In [31]:
df.query('trade == "S" & spd < 0')

,trade,name,spd,reason,market,qty,target,current,change,%change,active,xdate,amount


In [32]:
colx = ['trade', 'name', 'spd', 'reason', 'market', 'qty', 'target', 'current', 'change', '%change',
        'active', 'xdate']

In [33]:
file_name = "orders-log.csv"
output_file = os.path.join(dat_path, file_name)
god_file = os.path.join(god_path, file_name)
icd_file = os.path.join(icd_path, file_name)
osd_file = os.path.join(osd_path, file_name)

In [34]:
print(f"output_file : {output_file}") 
print(f"icd_file : {icd_file}") 
print(f"god_file : {god_file}") 
print(f"osd_file : {osd_file}") 

output_file : C:\Users\PC1\OneDrive\A5\Data\orders-log.csv
icd_file : C:\Users\PC1\iCloudDrive\Data\orders-log.csv
god_file : C:\Users\PC1\OneDrive\Imports\santisoontarinka@gmail.com - Google Drive\Data\orders-log.csv
osd_file : C:\Users\PC1\OneDrive\Documents\obsidian-git-sync\Data\orders-log.csv


In [35]:
df[colx].to_csv(output_file, header=True, index=False)
df[colx].to_csv(god_file, header=True, index=False)
df[colx].to_csv(icd_file, header=True, index=False)
df[colx].to_csv(osd_file, header=True, index=False)

### Convert column name to make it valid for df.query

In [37]:
df['percent'] = df['%change']

In [38]:
df.query('percent < -2')

,trade,name,spd,reason,market,qty,target,current,change,%change,active,xdate,amount,percent


In [39]:
df.query('percent >= 2.00')

,trade,name,spd,reason,market,qty,target,current,change,%change,active,xdate,amount,percent


In [40]:
df.loc[df.trade == 'B'].sort_values('spd',ascending=False)

,trade,name,spd,reason,market,qty,target,current,change,%change,active,xdate,amount,percent
1,B,CPF,2,NaN,SET50,5000,24.10,23.90,0.00,0.00,1,2025-08-29,120500.00,0.00
8,B,TFFIF,1,RD30Pct,SET,10000,6.00,5.95,-0.05,-0.83,1,2025-08-28,60000.00,-0.83
2,B,GVREIT,0,RND20Pct,SET,5000,6.20,6.20,0.00,0.00,1,2025-08-27,31000.00,0.00
5,B,ORI,0,RD70Pct,SET100,5000,2.20,2.20,0.00,0.00,1,2025-03-13,11000.00,0.00
7,B,RCL,0,RD35pct,SET100,3000,28.00,28.00,-0.25,-0.88,1,2025-08-22,84000.00,-0.88
6,B,PTT,-2,NaN,SET50,2500,31.00,31.50,-0.25,-0.79,1,2025-03-06,77500.00,-0.79
3,B,JMART,-9,RD80Pct,SET50,3200,8.00,8.45,0.10,1.20,1,2025-05-09,25600.00,1.20
4,B,JMT,-9,RD75pct,SET50,2800,11.00,11.90,0.10,0.85,1,2025-08-26,30800.00,0.85
0,B,AIMIRT,-12,RND15pct,SET,2500,9.10,9.70,0.00,0.00,1,2025-08-19,22750.00,0.00
10,B,WHART,-23,RD30Pct,SET,5000,8.00,9.15,0.00,0.00,0,2025-08-18,40000.00,0.00


### Sell recently purchased stocks if profitable

In [42]:
#pd.set_option('display.max_rows', 40)
sql = """SELECT name, qty, price AS cost, date 
FROM buys B 
JOIN stocks S 
ON stock_id = S.id 
WHERE status = 'Active'
ORDER BY name, date"""
buys = pd.read_sql(sql, conpf)
buys.shape

(112, 4)

In [43]:
colu = 'name date_x qty cost price target %change %profit cst_amt mkt_amt difa'.split()
buys_price = pd.merge(buys, prices, on='name', how='inner')
buys_price['diff'] = buys_price['price'] - buys_price['cost']
buys_price['difa'] = buys_price['diff'] * buys_price['qty']
buys_price['%profit'] = round(buys_price['difa'] / (buys_price['qty'] * buys_price['cost']) * 100,2)

In [44]:
buys_price['cst_amt'] = buys_price['qty'] * buys_price['cost']
buys_price['mkt_amt'] = buys_price['qty'] * buys_price['price']
buys_price['target'] = buys_price['cost'] * 1.1

In [45]:
laggards = buys_price.loc[buys_price.cost > buys_price.price,colu]
laggards.shape

(107, 11)

In [46]:
watch_list = buys_price.loc[buys_price.cost <= buys_price.price,colu]
watch_list.sort_values('%profit',ascending=False).style.format(format_dict)

,name,date_x,qty,cost,price,target,%change,%profit,cst_amt,mkt_amt,difa
24,DIF,2025-04-09,"5,000",7.50,8.20,8.25,0.00,9.33,"37,500.00","41,000.00","3,500.00"
102,TVO,2025-03-12,"4,000",23.10,25.00,25.41,1.63,8.23,"92,400.00","100,000.00","7,600.00"
111,WHART,2025-07-08,"5,000",8.80,9.15,9.68,0.00,3.98,"44,000.00","45,750.00","1,750.00"
101,TVO,2024-11-06,"4,000",24.90,25.00,27.39,1.63,0.40,"99,600.00","100,000.00",400.00
77,RCL,2025-08-18,"3,000",28.00,28.00,30.80,-0.88,0.00,"84,000.00","84,000.00",0.00


In [47]:
current_path = os.getcwd()
base_path = os.path.dirname(current_path)
ruby_path = os.path.join(base_path, "Ruby")
print(f"Ruby path (ruby_path): {ruby_path}") 

Ruby path (ruby_path): C:\Users\PC1\OneDrive\A5\Ruby


In [48]:
file_name = "watch-list-sell.xlsx"
output_file = os.path.join(ruby_path, file_name)
watch_list[colu].sort_values('%profit',ascending=False).to_excel(output_file, header=True, index=False)

In [49]:
float_formatter = "฿{:,.2f}"
float_formatter.format(watch_list.difa.sum())

'฿13,250.00'

In [50]:
sell_list = buys_price.loc[buys_price['%profit'] >= 10.00,colu]
sell_list.sort_values('%profit',ascending=False).style.format(format_dict)

,name,date_x,qty,cost,price,target,%change,%profit,cst_amt,mkt_amt,difa


In [51]:
float_formatter = "฿{:,.2f}"
float_formatter.format(sell_list.cst_amt.sum())

'฿0.00'

In [52]:
float_formatter = "฿{:,.2f}"
float_formatter.format(sell_list.mkt_amt.sum())

'฿0.00'

In [53]:
float_formatter = "฿{:,.2f}"
float_formatter.format(sell_list.difa.sum())

'฿0.00'

### Profit per Month

In [55]:
year = 2025
month = 8
sql = '''
SELECT name, YEAR(sells.date) AS sell_year, MONTH(sells.date) AS sell_month, 
       sells.price AS sell_price, buys.price AS buy_price, (sells.price - buys.price) AS diff, 
       qty, (sells.price * qty) AS sell_amt, (buys.price * qty) AS buy_amt, 
       (sells.price - buys.price) * qty AS gross, 
       ROUND((sells.price - buys.price)/buys.price*100,2) AS pct, days
FROM sells JOIN buys ON sells.buy_id = buys.id
JOIN stocks ON buys.stock_id = stocks.id
WHERE YEAR(sells.date) = %s AND MONTH(sells.date) = %s
ORDER BY sells.date DESC, name'''
sql = sql % (year, month)
sells_df = pd.read_sql(sql, conpf)
print(sql)


SELECT name, YEAR(sells.date) AS sell_year, MONTH(sells.date) AS sell_month, 
       sells.price AS sell_price, buys.price AS buy_price, (sells.price - buys.price) AS diff, 
       qty, (sells.price * qty) AS sell_amt, (buys.price * qty) AS buy_amt, 
       (sells.price - buys.price) * qty AS gross, 
       ROUND((sells.price - buys.price)/buys.price*100,2) AS pct, days
FROM sells JOIN buys ON sells.buy_id = buys.id
JOIN stocks ON buys.stock_id = stocks.id
WHERE YEAR(sells.date) = 2025 AND MONTH(sells.date) = 8
ORDER BY sells.date DESC, name


In [56]:
sells_df.style.format(format_dict)

,name,sell_year,sell_month,sell_price,buy_price,diff,qty,sell_amt,buy_amt,gross,pct,days
0,ORI,2025,8,2.30,2.20,0.10,"5,000","11,500.00","11,000.00",500.00,4.55,10
1,MCS,2025,8,8.40,7.30,1.10,"6,000","50,400.00","43,800.00","6,600.00",15.07,187
2,CPF,2025,8,24.30,24.10,0.20,"5,000","121,500.00","120,500.00","1,000.00",0.83,1
3,CPF,2025,8,24.30,24.10,0.20,"4,000","97,200.00","96,400.00",800.00,0.83,1
4,CPF,2025,8,24.30,24.10,0.20,"4,000","97,200.00","96,400.00",800.00,0.83,2
5,GVREIT,2025,8,6.70,6.10,0.60,"6,000","40,200.00","36,600.00","3,600.00",9.84,686
6,RCL,2025,8,31.00,27.75,3.25,"3,000","93,000.00","83,250.00","9,750.00",11.71,1023
7,JMART,2025,8,9.40,9.25,0.15,"3,200","30,080.00","29,600.00",480.00,1.62,1
8,ORI,2025,8,2.06,1.85,0.21,"10,000","20,600.00","18,500.00","2,100.00",11.35,129
9,JMT,2025,8,11.60,10.50,1.10,"2,800","32,480.00","29,400.00","3,080.00",10.48,79


In [57]:
print(f"Monthly Profit Amount: {sells_df.gross.sum():,.2f}")

Monthly Profit Amount: 32,010.00


In [58]:
file_name = "monthly-sales.csv"
output_file = os.path.join(dat_path, file_name)
god_file = os.path.join(god_path, file_name)
icd_file = os.path.join(icd_path, file_name)
osd_file = os.path.join(osd_path, file_name)

In [59]:
print(f"output_file : {output_file}") 
print(f"icd_file : {icd_file}") 
print(f"god_file : {god_file}") 
print(f"osd_file : {osd_file}") 

output_file : C:\Users\PC1\OneDrive\A5\Data\monthly-sales.csv
icd_file : C:\Users\PC1\iCloudDrive\Data\monthly-sales.csv
god_file : C:\Users\PC1\OneDrive\Imports\santisoontarinka@gmail.com - Google Drive\Data\monthly-sales.csv
osd_file : C:\Users\PC1\OneDrive\Documents\obsidian-git-sync\Data\monthly-sales.csv


In [60]:
sells_df.to_csv(output_file, header=True, index=False)
sells_df.to_csv(god_file, header=True, index=False)
sells_df.to_csv(icd_file, header=True, index=False)
sells_df.to_csv(osd_file, header=True, index=False)

In [61]:
sells_df['profit_per_day'] = (sells_df['gross'] / sells_df['buy_amt']) / sells_df['days'] * 100
sells_df.style.format(format_dict)

,name,sell_year,sell_month,sell_price,buy_price,diff,qty,sell_amt,buy_amt,gross,pct,days,profit_per_day
0,ORI,2025,8,2.30,2.20,0.10,"5,000","11,500.00","11,000.00",500.00,4.55,10,0.454545
1,MCS,2025,8,8.40,7.30,1.10,"6,000","50,400.00","43,800.00","6,600.00",15.07,187,0.080580
2,CPF,2025,8,24.30,24.10,0.20,"5,000","121,500.00","120,500.00","1,000.00",0.83,1,0.829876
3,CPF,2025,8,24.30,24.10,0.20,"4,000","97,200.00","96,400.00",800.00,0.83,1,0.829876
4,CPF,2025,8,24.30,24.10,0.20,"4,000","97,200.00","96,400.00",800.00,0.83,2,0.414938
5,GVREIT,2025,8,6.70,6.10,0.60,"6,000","40,200.00","36,600.00","3,600.00",9.84,686,0.014338
6,RCL,2025,8,31.00,27.75,3.25,"3,000","93,000.00","83,250.00","9,750.00",11.71,1023,0.011448
7,JMART,2025,8,9.40,9.25,0.15,"3,200","30,080.00","29,600.00",480.00,1.62,1,1.621622
8,ORI,2025,8,2.06,1.85,0.21,"10,000","20,600.00","18,500.00","2,100.00",11.35,129,0.087995
9,JMT,2025,8,11.60,10.50,1.10,"2,800","32,480.00","29,400.00","3,080.00",10.48,79,0.132610


In [62]:
float_formatter = "฿{:,.2f}"
float_formatter.format(sells_df.gross.sum())

'฿32,010.00'

In [63]:
average_profit_percent = sells_df['profit_per_day'].mean()
print(f"Average Profit Percent: {average_profit_percent:.2f}%")

Average Profit Percent: 0.49%


In [64]:
#dt = date(2023, 11, 8)
dt = today
dt.strftime("%B %d, %Y")

'August 28, 2025'

In [65]:
print("Hello, {}! Today is {:%B %d, %Y}.".format("Alice", dt))

Hello, Alice! Today is August 28, 2025.


In [66]:
conlite.commit()
conlite.close()

In [67]:
# Get the current time
current_time = datetime.now()
# Format the time to remove millisAeconds
formatted_time = current_time.strftime("%Y-%m-%d %H:%M:%S")
print(formatted_time)

2025-08-28 21:01:15
